In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score
from gpytorch.mlls import ExactMarginalLogLikelihood
import gpytorch
import altair as alt
import matplotlib.pyplot as plt
import re
from scipy.stats import norm
from sklearn.cluster import KMeans

UTILITY

In [2]:
def plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration, label="target_value"):
    x = np.arange(len(candidates))  # Struktur-Index
    names = candidates["structure_name"].values

    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    plt.subplots_adjust(hspace=0.3)

    axs[0].plot(x, mean_np, color="C0", label="Predicted Mean")
    axs[0].fill_between(x, mean_np - std_np, mean_np + std_np, color="C0", alpha=0.3, label="Uncertainty (±1σ)")
    axs[0].set_ylabel("Predicted Mean")
    axs[0].set_title(f"Iteration {iteration}: GP Prediction (mean ± std)")
    axs[0].legend()
    axs[0].grid(alpha=0.3)

    axs[1].plot(x, ei_np, color="C1")
    axs[1].set_ylabel("Expected Improvement")
    axs[1].set_title("Expected Improvement over Candidates")
    axs[1].grid(alpha=0.3)

    axs[2].plot(x, pi_np, color="C2")
    axs[2].set_ylabel("Probability of Improvement")
    axs[2].set_xlabel("Candidate Structure Index")
    axs[2].set_title("Probability of Improvement")
    axs[2].grid(alpha=0.3)

    # Optional: Struktur-Namen als xticks (nur alle paar, sonst zu viele)
    if len(names) <= 30:
        axs[2].set_xticks(x)
        axs[2].set_xticklabels(names, rotation=90)
    else:
        step = max(1, len(names)//30)
        axs[2].set_xticks(x[::step])
        axs[2].set_xticklabels(names[::step], rotation=90)

    plt.tight_layout()
    plt.show()


In [3]:
def is_bin_column(col) -> bool:
    # numerische Spaltennamen erlauben
    if isinstance(col, (int, np.integer)):
        return True

    s = str(col)

    # rein numerischer Spaltenname: '0', '1', ...
    if s.isdigit():
        return True

    # bin_X oder bin_X_high / bin_X_low
    if re.fullmatch(r"bin_\d+(_high|_low)?", s):
        return True
    if re.fullmatch(r"bin_\d+", s): # 'bin_0', 'bin_1', ...
        return True

    return False

MODELL GP

In [4]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp(xt_train, yt_train, training_iterations=100):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(xt_train, yt_train, likelihood)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
    mll = ExactMarginalLogLikelihood(likelihood, model)

    losses = []

    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(xt_train)
        loss = -mll(output, yt_train)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    return model.eval(), likelihood.eval(), losses

DATEN EINLESEN & FILTERN

In [5]:
# dft Daten - beladungen, grid etc
dft_data1 = pd.read_csv('/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/DFT_Data_clean_06_10.csv')
dft_data2 = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/dft_fckin_clean_kond_64grid.csv")
dft_data_all =  pd.concat([dft_data1, dft_data2], ignore_index=True)

# Feature Daten - bins zu Vext, Vext+chem_res etc
#expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_chem_res_allTEMP_pressure_20b_exp.csv")
expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allcsv/Vext_allTEMP_64grid_20b.csv")

# Chem_res_bulk explizit - für additional feature 
chem_res = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv")

# Kombi aus obigem 
data = pd.merge(dft_data_all, expV_data, 'inner', on=["structure_name", "temperature_kelvin"])#, "pressure_bar"])
data = pd.merge(data, chem_res, 'inner', on=["structure_name", "temperature_kelvin", "pressure_bar"])
#feature_columns = [col for col in data.columns if is_bin_column(col)]
data = data[data.beladung_mol_per_kg > 0]

# wc(p, T) -> high
data_high = data[(data.temperature_kelvin == 400) & (data.pressure_bar == 1)]
data_high = data_high.drop_duplicates(subset=["structure_name", "temperature_kelvin", "pressure_bar"])
feature_columns_high = [col for col in data_high.columns if is_bin_column(col)]

# wc(p, T) -> low
data_low = data[(data.temperature_kelvin == 300) & (data.pressure_bar == 1)]
data_low = data_low.drop_duplicates(subset=["structure_name", "temperature_kelvin", "pressure_bar"])
feature_columns_low = [col for col in data_low.columns if is_bin_column(col)]

#duplicates = data[data.duplicated(subset=["structure_name", "temperature_kelvin", "pressure_bar"], keep=False)]
#print(duplicates[["structure_name", "temperature_kelvin", "pressure_bar"]])

print(data_high.shape == data_low.shape)
print(f"Data_HIGH: {data_high.shape}")
print(f"Data_LOW: {data_low.shape}")
#print(data_low)
#print(data_low.head())

True
Data_HIGH: (245, 43)
Data_LOW: (245, 43)


FEATURES/LABEL CUSTOM

In [6]:
add_features = False

data_high["beladung_pro_vol"] = (
    data_high["beladung_atoms"]
    .div(data_high["density_bulk"], axis=0)
    .div(data_high["volume_kubAng"], axis=0)
)

data_low["beladung_pro_vol"] = (
    data_low["beladung_atoms"]
    .div(data_low["density_bulk"], axis=0)
    .div(data_low["volume_kubAng"], axis=0)
)

data_high[feature_columns_high] = (
    data_high[feature_columns_high]
    .multiply(data_high["grid.dv"], axis=0)
    .div(data_high["volume_kubAng"], axis=0)
)

data_low[feature_columns_low] = (
    data_low[feature_columns_low]
    .multiply(data_low["grid.dv"], axis=0)
    .div(data_low["volume_kubAng"], axis=0)
)

additional_features12 = ["delta_p", "delta_T"]
additional_features = ["chem_potential_bulk_high", "chem_potential_bulk_low", "pressure_bar_high", "pressure_bar_low", "temperature_kelvin_high", "temperature_kelvin_low"]

merged = pd.merge(
    #data_high[["structure_name", "beladung_pro_vol"]],
    #data_low[["structure_name", "beladung_pro_vol"]],
    data_high,#[cols],
    data_low,#[cols],
    on="structure_name",
    suffixes=("_high", "_low")
)

# wc aus den beiden Zuständen high/low
merged["working_capacity"] = (merged["beladung_pro_vol_high"] - merged["beladung_pro_vol_low"]).abs()
merged["delta_T"] = (merged["temperature_kelvin_high"] - merged["temperature_kelvin_low"]).abs()
merged["delta_p"] = (merged["pressure_bar_high"] - merged["pressure_bar_low"]).abs()
feature_columns = [col for col in merged.columns if is_bin_column(col)]

if add_features:
    feature_columns += additional_features

merged["working_capacity"] = pd.to_numeric(merged["working_capacity"], errors="coerce")
print(merged["working_capacity"])
print(feature_columns)

0      41.390929
1      37.639063
2      45.146102
3      41.572863
4      55.500263
         ...    
240    16.440841
241     9.519076
242    35.962617
243    38.921185
244    67.293831
Name: working_capacity, Length: 245, dtype: float64
['bin_0_high', 'bin_1_high', 'bin_2_high', 'bin_3_high', 'bin_4_high', 'bin_5_high', 'bin_6_high', 'bin_7_high', 'bin_8_high', 'bin_9_high', 'bin_10_high', 'bin_11_high', 'bin_12_high', 'bin_13_high', 'bin_14_high', 'bin_15_high', 'bin_16_high', 'bin_17_high', 'bin_18_high', 'bin_19_high', 'bin_0_low', 'bin_1_low', 'bin_2_low', 'bin_3_low', 'bin_4_low', 'bin_5_low', 'bin_6_low', 'bin_7_low', 'bin_8_low', 'bin_9_low', 'bin_10_low', 'bin_11_low', 'bin_12_low', 'bin_13_low', 'bin_14_low', 'bin_15_low', 'bin_16_low', 'bin_17_low', 'bin_18_low', 'bin_19_low']


NORMALIZE

In [7]:
normalize_feature = True
normalize_labels = True

FOLD - TRAINING - PREDICTION

In [8]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

label = "working_capacity"
#X = data[feature_columns].values 
X = merged[feature_columns].values
#y = data[label].values 
y = merged[label].values 

ids = data.index.values

split_info = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
    x_train = torch.tensor(X[train_idx], dtype=torch.float64)
    y_train = torch.tensor(y[train_idx], dtype=torch.float64)
    x_test = torch.tensor(X[test_idx], dtype=torch.float64)
    y_test = torch.tensor(y[test_idx], dtype=torch.float64)

    train_ids = ids[train_idx]
    test_ids = ids[test_idx]

    #test_df = data.iloc[test_idx].copy()
    test_df = merged.iloc[test_idx].copy()
    test_df["fold"] = fold

    if normalize_feature:
        feature_transformer = MinMaxScaler()
        feature_transformer.fit(x_train)
        xt_train = torch.tensor(feature_transformer.transform(x_train), dtype=torch.float64)
        xt_test = torch.tensor(feature_transformer.transform(x_test), dtype=torch.float64) #*2
    else:
        xt_train = x_train
        xt_test = x_test

    # Label-Normalisierung
    if normalize_labels:
        label_transformer = MinMaxScaler()  # oder StandardScaler()
        label_transformer.fit(y_train.unsqueeze(1))
        yt_train = torch.tensor(label_transformer.transform(y_train.unsqueeze(1)).flatten(), dtype=torch.float64)
        yt_test = torch.tensor(label_transformer.transform(y_test.unsqueeze(1)).flatten(), dtype=torch.float64)
    else:
        yt_train = y_train
        yt_test = y_test

    # Training
    model, likelihood, losses = train_gp(xt_train, yt_train, training_iterations=200)

    # Prediction
    with torch.no_grad():
        prediction = model(xt_test)
        inverse_transformed_prediction = label_transformer.inverse_transform(
            prediction.mean.unsqueeze(1)
        ).squeeze()
        inverse_transformed_prediction = np.where(
            inverse_transformed_prediction > 0, inverse_transformed_prediction, 0
        )

    # Ergebnisse
    test_df[f"{label}_pred"] = inverse_transformed_prediction
    test_df["abs_rel_deviation"] = np.abs(
        (test_df[label] - test_df[f"{label}_pred"]) / test_df[label] * 100
    )

    split_info.append(test_df)

results = pd.concat(split_info, ignore_index=True)
#results
# results.to_csv("GP_results_beladung_pro_vol_400K_0.1bar_customFeatures.csv", index=False)

ALLE DATEN

In [9]:
print(f"R²                        : {r2_score(results[label], results[f'{label}_pred']):.4f}")
print(f"Median APE                : {results['abs_rel_deviation'].median():.2f}%")
print(f"Mean APE                  : {results['abs_rel_deviation'].mean():.2f}%")
print(f"Final Loss                : {losses[-1]:.4f}")

count = (results['abs_rel_deviation'] > 20).sum()
print(f"Abs rel dev > 20%         : {count} out of {len(results)}")
print(f"Max abs rel dev           : {results['abs_rel_deviation'].max():.2f}%")


R²                        : 0.5899
Median APE                : 18.11%
Mean APE                  : 10686255.70%
Final Loss                : -0.9244
Abs rel dev > 20%         : 113 out of 245
Max abs rel dev           : 2531530943.45%


EINZELNE FOLDS

In [10]:
for fold, group in results.groupby("fold"):
    print(f"\nFold {fold}")
    print(f"R²           : {r2_score(group[label], group[f'{label}_pred']):.4f}")
    print(f"Median APE   : {group['abs_rel_deviation'].median():.2f}%")
    print(f"Mean APE     : {group['abs_rel_deviation'].mean():.2f}%")
    print(f"Max ARD      : {group['abs_rel_deviation'].max():.2f}%")
    print(f"Final Loss   : {losses[-1]:.4f}")



Fold 1
R²           : 0.6016
Median APE   : 17.36%
Mean APE     : 23.05%
Max ARD      : 105.84%
Final Loss   : -0.9244

Fold 2
R²           : 0.7138
Median APE   : 15.93%
Mean APE     : 871.42%
Max ARD      : 19914.21%
Final Loss   : -0.9244

Fold 3
R²           : 0.4998
Median APE   : 21.74%
Mean APE     : 69.33%
Max ARD      : 588.55%
Final Loss   : -0.9244

Fold 4
R²           : 0.7265
Median APE   : 14.46%
Mean APE     : 55.82%
Max ARD      : 612.50%
Final Loss   : -0.9244

Fold 5
R²           : 0.8028
Median APE   : 15.79%
Mean APE     : 5088.69%
Max ARD      : 126662.49%
Final Loss   : -0.9244

Fold 6
R²           : 0.3828
Median APE   : 18.19%
Mean APE     : 93.75%
Max ARD      : 1682.82%
Final Loss   : -0.9244

Fold 7
R²           : 0.4261
Median APE   : 26.06%
Mean APE     : 65.94%
Max ARD      : 476.69%
Final Loss   : -0.9244

Fold 8
R²           : 0.5256
Median APE   : 17.95%
Mean APE     : 66.97%
Max ARD      : 1078.46%
Final Loss   : -0.9244

Fold 9
R²           : 0.6428


PARITY PLOT - ALT

In [11]:
alt.Chart(results).mark_circle(size=60).encode(
    x=label,
    y=f"{label}_pred",
    color="fold:N",
    tooltip=["structure_name", label, f"{label}_pred", "abs_rel_deviation", "fold"]
).interactive()

alt.Chart(...)

In [12]:
alt.Chart(results[results["fold"] == 1]).mark_circle(size=60).encode(
    x=label,
    y=f"{label}_pred",
    color="fold:N",
    tooltip=["structure_name", label, f"{label}_pred", "abs_rel_deviation", "fold"]
).interactive()


alt.Chart(...)

DATA ANALYSIS

In [13]:
"""
high_dev_names = results.loc[results["abs_rel_deviation"] > 20, "structure_name"].tolist()
filtered_results = results[results["structure_name"].isin(high_dev_names)]
filtered_results

filter_res = results.sort_values(by="beladung_pro_vol", ascending=False)

filter_res.head()
"""

'\nhigh_dev_names = results.loc[results["abs_rel_deviation"] > 20, "structure_name"].tolist()\nfiltered_results = results[results["structure_name"].isin(high_dev_names)]\nfiltered_results\n\nfilter_res = results.sort_values(by="beladung_pro_vol", ascending=False)\n\nfilter_res.head()\n'

BAYESIAN OPTIMIZATION

In [14]:
from torch.distributions import Normal

def af_log_expIm(mean, var, best_f, xi=0.01):
    """Logarithmic Expected Improvement acquisition function."""

    std = torch.sqrt(var)
    std_safe = torch.clamp(std, min=1e-9)  # Avoid division by zero
    z = (mean - best_f - xi) / std_safe
    normal = Normal(torch.zeros_like(z), torch.ones_like(z))
    cdf = normal.cdf(z)
    pdf = torch.exp(normal.log_prob(z))

    ei = std * (z * cdf + pdf)

    ei_safe = torch.clamp(ei, min=1e-9)  # Avoid log(0)
    log_ei = torch.log(ei_safe)
    return log_ei
    


In [16]:
#candidates = data.copy() # zunächst gefilteret, später alle Daten
patience = 10
candidates = merged.copy() # zunächst gefilteret, später alle Daten
"""
n_initial = 1 # Anzahl der initialen Trainingspunkte
initial_indices = candidates.nsmallest(n_initial, label).index # hier geht auch random
"""
########################
n_initial = 10
X = candidates[feature_columns]     # deine Feature-Spalten

km = KMeans(n_clusters=n_initial, n_init='auto')
labels_km = km.fit_predict(X)

# pro Cluster den Punkt nehmen, der dem Zentrum am nächsten liegt
initial_indices = (
    X.groupby(labels_km)
     .apply(lambda g: g.index[(g - km.cluster_centers_[g.name]).pow(2).sum(axis=1).idxmin()])
     .tolist()
)
########################
print(f"Initial training points:")
for idx in initial_indices:
    print(f"  Index {idx}, Structure {candidates.loc[idx, 'structure_name']}, {label}: {candidates.loc[idx, label]:.4f}")

# Transfer from candidates to selection
selected = candidates.loc[initial_indices]
candidates = candidates.drop(initial_indices)
best = [selected[label].max()]

for i in range(100):
    if len(best) >= patience:
        if len(np.unique(best[-patience:])) == 1:
            print(f"Early stopping at iteration {i} due to no improvement in the last {patience} iterations.")
            break
    
    feature_transoformer = MinMaxScaler()
    label_transformer = MinMaxScaler()

    train_x = torch.tensor(feature_transoformer.fit_transform(selected[feature_columns].values))
    train_y = torch.tensor(label_transformer.fit_transform(selected[[label]].values)).flatten()

    test_x = torch.tensor(feature_transoformer.transform(candidates[feature_columns].values))

    model, likelihood, _ = train_gp(train_x, train_y, 250)
    with torch.no_grad():
        prediction = model(test_x)
        mean, var = prediction.mean, prediction.variance
    
    best_f = train_y.max()

    log_ei = af_log_expIm(mean, var, best_f, 0.01 * best_f)

    # Select the candidate with the highest acquisition value
    index = torch.argmax(log_ei).item()
    best.append(selected[label].max())
    print(f"Iteration: {i}, Current Best: {selected[label].max():.2e}")
    selected = pd.concat([selected, candidates.iloc[[index]]])
    canidates = candidates.drop(candidates.index[index])

print(f"Best Value after {len(best)} iterations: {best[-1]}")

mean_np = mean.detach().cpu().numpy().flatten()
var_np = var.detach().cpu().numpy().flatten()
std_np = np.sqrt(var_np)
ei_np = torch.exp(log_ei).detach().cpu().numpy().flatten()

# --- Probability of Improvement ---
best_f = train_y.max().item()
z = (mean_np - best_f) / std_np
pi_np = norm.cdf(z)


# --- Plot am Ende ---
plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration=i, label=label)
    

IndexError: index 213 is out of bounds for axis 0 with size 50

In [ ]:
plt.plot(best)
plt.hlines(merged[label].max(), 0, len(best)-1, colors='r', linestyles='dashed')
plt.ylabel(f"Label: {label}")
plt.xlabel("Iterations")